In [83]:
!pip -q install transformers accelerate

In [84]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, set_seed

In [85]:
model_name = "Qwen/Qwen2.5-1.5B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    torch_dtype=torch.float16
)
model.eval()

Loading weights:   0%|          | 0/338 [00:00<?, ?it/s]

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 1536)
    (layers): ModuleList(
      (0-27): 28 x Qwen2DecoderLayer(
        (self_attn): Qwen2Attention(
          (q_proj): Linear(in_features=1536, out_features=1536, bias=True)
          (k_proj): Linear(in_features=1536, out_features=256, bias=True)
          (v_proj): Linear(in_features=1536, out_features=256, bias=True)
          (o_proj): Linear(in_features=1536, out_features=1536, bias=False)
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=1536, out_features=8960, bias=False)
          (up_proj): Linear(in_features=1536, out_features=8960, bias=False)
          (down_proj): Linear(in_features=8960, out_features=1536, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): Qwen2RMSNorm((1536,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((1536,), eps=1e-06)
      )
    )
    (norm): Qwen2RMSNorm((1536,), eps=1e-06)
    (rotar

In [86]:
prompt = """
Explain photosynthesis for a non-scientist.

Return ONLY this template and fill it:

- Bullet 1:
- Bullet 2:
- Bullet 3:
- Bullet 4:
- Bullet 5:

Rules:
- Replace each bullet with one short sentence (<= 12 words).
- Keep exactly 5 lines starting with "- ".
- No extra text before or after.
""".strip()

In [87]:
def generate(prompt, do_sample=False, temperature=0.7, top_p=0.9, max_new_tokens=120, seed=0):
    set_seed(seed)
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    out = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        do_sample=do_sample,
        temperature=temperature,
        top_p=top_p,
        repetition_penalty=1.05,
        pad_token_id=tokenizer.eos_token_id
    )

    return tokenizer.decode(out[0], skip_special_tokens=True)

In [88]:
print("=== GREEDY ===\n")
print(generate(prompt, do_sample=False))

=== GREEDY ===

Explain photosynthesis for a non-scientist.

Return ONLY this template and fill it:

- Bullet 1:
- Bullet 2:
- Bullet 3:
- Bullet 4:
- Bullet 5:

Rules:
- Replace each bullet with one short sentence (<= 12 words).
- Keep exactly 5 lines starting with "- ".
- No extra text before or after. - Photosynthesis is the process by which plants make their own food using sunlight.
- It involves converting carbon dioxide and water into glucose and oxygen.
- The glucose provides energy for the plant, while oxygen is released into the air.
- This process helps sustain life on Earth by producing oxygen.
- Plants use sunlight to power this process through a series of chemical reactions. - Sunlight is absorbed by chlorophyll in leaves.
- Water from the soil is split into hydrogen and oxygen.
- Carbon dioxide from the air combines with hydrogen to form glucose.
- Oxygen is released as a waste product, but


In [89]:
print("\n=== SAMPLING RUNS (temp=0.7, top_p=0.9) ===\n")
for i in range(3):
    print(f"\nRun {i}")
    print(generate(prompt, do_sample=True, temperature=0.7, top_p=0.9, seed=i))


=== SAMPLING RUNS (temp=0.7, top_p=0.9) ===


Run 0
Explain photosynthesis for a non-scientist.

Return ONLY this template and fill it:

- Bullet 1:
- Bullet 2:
- Bullet 3:
- Bullet 4:
- Bullet 5:

Rules:
- Replace each bullet with one short sentence (<= 12 words).
- Keep exactly 5 lines starting with "- ".
- No extra text before or after. - Photosynthesis is when plants use sunlight to make their food.
- The process happens in the leaves of plants.
- It uses water, air, and sunlight as ingredients.
- The end product is sugar and oxygen gas.
- This process helps plants grow and give us food. - Plants make their own food using sunlight, water, and air.
- The green part of the plant called chlorophyll does the job.
- They turn carbon dioxide from the air into sugar.
- Oxygen is a byproduct that plants release into the air.
- This is how plants help clean the air and make

Run 1
Explain photosynthesis for a non-scientist.

Return ONLY this template and fill it:

- Bullet 1:
- Bullet 2:
-